In [19]:
corpus = [
           'I have for a while now really only used Mint for an aggregator of my transactions and balances due to not using exact numbers but now they appear to have taken that away from the account balances too. Mint is good for people who are bad at budgeting but there are a lot better ways to budget',
           'Why does the new app round up the account balance rather than show the exact amount now? The balance on every account is to the next dollar higher rather than to the exact penny the version before.  Accuracy is important.   This is a bad feature on the new update',
           'The recent upgrade doesn’t show actual amounts in accounts. It rounds to the nearest dollar.',
           'The app does what its supposed to do. Need an uplift in the UI. Basically the colors look really dull. Also need options to create and manage new categories.',
           'Two hours of trying to delete this account and it is not working. I followed the online instructions but the link did not work, tried from inside the account, talked to chat support but she couldn’t figure it out, now I am on the phone AGAIN hoping for a miracle which is unlikely since I am told there are many people having problems with the app',
           'I have only had the app for 2 days, and it is already not working. I am trying to add a transaction, but it says “Unable to connect to server.” Everything else is working fine on my phone.',
           'Used to work great. Now it says “uhoh can’t connect to the server” even though it obviously can since it loads new transactions. Much much slower than before. Can’t see accounts because pressing anywhere in that area automatically loads their new net worth chart'
]

In [20]:
import nltk
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [22]:
'''
Code to preprocess the data, break the reviews into sentences 
in case words > max_allowed_words in a sentence OR 
number of sentences > max allowed sentences in a sentence
'''


#https://github.com/summanlp/textrank
from summa import keywords
from summa.summarizer import summarize

sentences=[]
max_words_in_sentence= 50
max_lines_in_sentence= 3

summarized_sentences=[]

for review in corpus:
    line_count = len(tokenizer.tokenize(review))
    temp_sentence=[]
    
    if(line_count>=max_lines_in_sentence):
        
        temp=summarize(review,2/line_count).split('\n')
        temp_sentence.extend(temp)
    else:
        temp_sentence.append(review)

    for sentence in temp_sentence:
        word_count = len(sentence.split())
        if(word_count>=max_words_in_sentence):
            summarized_sentences.extend(summarize(review,max_words_in_sentence).split('\n'))
        else:
            summarized_sentences.append(sentence)
                    
print(summarized_sentences)        
print(len(summarized_sentences))

['I have for a while now really only used Mint for an aggregator of my transactions and balances due to not using exact numbers but now they appear to have taken that away from the account balances too.', 'Mint is good for people who are bad at budgeting but there are a lot better ways to budget', 'Why does the new app round up the account balance rather than show the exact amount now?', 'The balance on every account is to the next dollar higher rather than to the exact penny the version before.', 'The recent upgrade doesn’t show actual amounts in accounts. It rounds to the nearest dollar.', 'Need an uplift in the UI.', 'Also need options to create and manage new categories.', 'Two hours of trying to delete this account and it is not working.', 'I followed the online instructions but the link did not work, tried from inside the account, talked to chat support but she couldn’t figure it out, now I am on the phone AGAIN hoping for a miracle which is unlikely since I am told there are man

In [46]:
#https://github.com/UKPLab/sentence-transformers
num_clusters = 3
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
corpus_embeddings = embedder.encode(summarized_sentences)
from sklearn.cluster import KMeans

clustering_model = KMeans(n_clusters=num_clusters)
clustering_model
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

for i in range(len(summarized_sentences)):
    print(str(summarized_sentences[i])+" -- "+str(cluster_assignment[i]))


I have for a while now really only used Mint for an aggregator of my transactions and balances due to not using exact numbers but now they appear to have taken that away from the account balances too. -- 2
Mint is good for people who are bad at budgeting but there are a lot better ways to budget -- 2
Why does the new app round up the account balance rather than show the exact amount now? -- 2
The balance on every account is to the next dollar higher rather than to the exact penny the version before. -- 2
The recent upgrade doesn’t show actual amounts in accounts. It rounds to the nearest dollar. -- 2
Need an uplift in the UI. -- 0
Also need options to create and manage new categories. -- 0
Two hours of trying to delete this account and it is not working. -- 1
I followed the online instructions but the link did not work, tried from inside the account, talked to chat support but she couldn’t figure it out, now I am on the phone AGAIN hoping for a miracle which is unlikely since I am told

In [47]:
clustered_sentences = {new_list: [] for new_list in range(num_clusters)} 
for i in range(len(cluster_assignment)):
    temp_list= clustered_sentences[cluster_assignment[i]]
    temp_list.append(summarized_sentences[i])
    clustered_sentences[cluster_assignment[i]]=temp_list
    
print(clustered_sentences)

{0: ['Need an uplift in the UI.', 'Also need options to create and manage new categories.'], 1: ['Two hours of trying to delete this account and it is not working.', 'I followed the online instructions but the link did not work, tried from inside the account, talked to chat support but she couldn’t figure it out, now I am on the phone AGAIN hoping for a miracle which is unlikely since I am told there are many people having problems with the app', 'I have only had the app for 2 days, and it is already not working. I am trying to add a transaction, but it says “Unable to connect to server.” Everything else is working fine on my phone.', 'Now it says “uhoh can’t connect to the server” even though it obviously can since it loads new transactions.'], 2: ['I have for a while now really only used Mint for an aggregator of my transactions and balances due to not using exact numbers but now they appear to have taken that away from the account balances too.', 'Mint is good for people who are bad

In [49]:
nltk.download('punkt')
nltk.download('stopwords')
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package punkt to /Users/rjain13/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rjain13/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import networkx as nx


word_embeddings = {}
f = open('./glove.6B/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()


In [75]:
#https://github.com/rohitkgp34/Text-Summarization
for var in range(num_clusters):
    clean_sentences = [s.lower() for s in clustered_sentences[var]]
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    
    print(clean_sentences)
    print()
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        
        sentence_vectors.append(v)
    
    sim_mat = np.zeros([len(clustered_sentences[var]), len(clustered_sentences[var])])
    for i in range(len(clustered_sentences[var])):
        for j in range(len(clustered_sentences[var])):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)

    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(clustered_sentences[var])), reverse=True)

    for i in range(2):
        print(i,":",ranked_sentences[i][1])

    print()
    print("----------********----------")
    print()
    


['need uplift ui.', 'also need options create manage new categories.']

0 : Need an uplift in the UI.
1 : Also need options to create and manage new categories.

----------********----------

['two hours trying delete account working.', 'followed online instructions link work, tried inside account, talked chat support couldn’t figure out, phone hoping miracle unlikely since told many people problems app', 'app 2 days, already working. trying add transaction, says “unable connect server.” everything else working fine phone.', 'says “uhoh can’t connect server” even though obviously since loads new transactions.']

0 : I followed the online instructions but the link did not work, tried from inside the account, talked to chat support but she couldn’t figure it out, now I am on the phone AGAIN hoping for a miracle which is unlikely since I am told there are many people having problems with the app
1 : I have only had the app for 2 days, and it is already not working. I am trying to add a tr